In [9]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
# Load dataset
dataset = pd.read_csv('df_model.csv', index_col=None)


In [11]:
dataset

,Airline,Top/Base,Category,Country,Income,Index,new_salary_USD_GROSS_2023
0,British Airways Mainline,CaptMax,Legacy,United Kingdom,44480.0,61.5,149533.830450
1,British Airways Mainline,CaptMin,Legacy,United Kingdom,44480.0,61.5,72408.066480
2,Cargo Air,CaptMax,Cargo,Bulgaria,NaN,40.5,59388.144077
3,Cargo Air,CaptMin,Cargo,Bulgaria,NaN,40.5,51964.626068
4,DHL Air UK,CaptMax,Cargo,United Kingdom,44480.0,61.5,141808.605960
...,...,...,...,...,...,...,...
87,Swiss International Air Lines,CaptMin,Legacy,Switzerland,90600.0,114.2,101552.354607
88,Atlantic Airways,CaptMax,Regional,Iceland,63460.0,83.3,80748.881408
89,Atlantic Airways,CaptMin,Regional,Iceland,63460.0,83.3,51162.359864
90,Exxaero,CaptMax,Fractional/Corporate,Netherlands,55200.0,68.6,72790.458000


In [13]:
dataset.columns = ["Airline", "Top/Base", "Category", "Country", "Income", "Index", "y"]
# Remove rows with NaN values
dataset = dataset.dropna()

# Split dataset into features and target variable
X = dataset.drop('y', axis=1)
y = dataset['y']

# Normalize data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define XGBoost parameters
params = {
    'max_depth': 3,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'objective': 'binary:logistic'
}

# Train XGBoost model on training set
xgboost_model = xgb.XGBClassifier(**params)
xgboost_model.fit(X_train, y_train)

# Evaluate model performance on test set
test_preds = xgboost_model.predict(X_test)
test_accuracy = np.sum(test_preds == y_test) / len(y_test)
print('Test set accuracy:', test_accuracy)

ValueError: could not convert string to float: 'British Airways Mainline'